In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
#from statsmodels.tsa.vector_ar.var_model import VARY
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime

from joblib import Parallel, delayed

import scipy
import importlib
from itertools import combinations
import forestplot as fp
import statsmodels.api as sm

In [ ]:
# sites = ['MCRI', 'MCW', 'UMHC', 'UNMC', 'UofU', 'UTHSCSA', 'KUMC', 'UTSW', 'UIOWA', 'UPITT', 'IUR']
# for site in sites:
#     try:
#         myco.get_onsetdata(site=site)
#         myco.get_vitaldata(site=site)
#         myco.get_labdata(site=site)
#         myco.get_meddata(site=site)
#         myco.lab = None
#         myco.med = None
#         myco.vital = None
#         datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
#         with open(datafolder+'myco_tmp_'+site+'.pkl', "wb") as f:
#              pickle.dump(myco, f)
#     except:
#         pass

In [ ]:
# sites = ['MCRI', 'MCW', 'UMHC', 'UNMC', 'UofU', 'UTHSCSA', 'KUMC', 'UTSW', 'UIOWA', 'UPITT', 'IUR']
# for site in sites:
#     try:
#         datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
#         import mycorr
#         importlib.reload(mycorr)
#         myco = mycorr.mycorr()
#         with open(datafolder+'myco_tmp_'+site+'.pkl', "rb") as f:
#             myco2 = pickle.load(f)
#         myco.copy(myco2)
# #        myco.get_onsetdata(site=site)
# #        myco.get_vitaldata(site=site)
# #        myco.get_labdata(site=site)
#         myco.get_meddata(site=site)
#         myco.lab = None
#         myco.med = None
#         myco.vital = None
#         datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
#         with open(datafolder+'myco_tmp_'+site+'.pkl', "wb") as f:
#              pickle.dump(myco, f)
#     except:
#         pass

In [ ]:
# site = 'UMHC'
# datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
# import mycorr
# importlib.reload(mycorr)
# myco = mycorr.mycorr()
# with open(datafolder+'myco2_tmp_'+site+'.pkl', "rb") as f:
#     myco2 = pickle.load(f)
# myco.copy(myco2)

In [ ]:
#mycoX = myco.calculate_site(site = 'KUMC')

In [ ]:
#mycoX.calculate_corr2()

In [ ]:
site = 'UMHC'
datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
import mycorr2
importlib.reload(mycorr2)
myco = mycorr2.mycorr2()
with open(datafolder+'myco2_tmp_'+site+'.pkl', "rb") as f:
    myco2 = pickle.load(f)
myco.copy(myco2)

In [ ]:
mycoX = myco.calculate_site('UMHC')

In [ ]:
myco.calculate_all()

In [ ]:
myco.calculate_all2(loinc2='2823-3', loinc3='FLAG', prefix='myco2_corr4_', plot_prefix="plot4_")

In [ ]:
mycoX.pcorr_custom()              

In [ ]:
myco.calculate_site('UMHC')

In [ ]:
mycoX = myco.calculate_site('UMHC')

In [ ]:
mycoX.res_df3

In [ ]:
mycoX.one_table

In [ ]:
dfmlr = pd.read_csv('mlr6_cofounder3.csv')

dfmlr['rho'] = -dfmlr['pdiff']
dfmlr['95ci_low'] = -(dfmlr['r']-dfmlr['CI95%0down'])
dfmlr['95ci_high'] = -(dfmlr['r']-dfmlr['CI95%0up'])
dfmlr['target'] = dfmlr['covar']
dfmlr['pvalue'] = (~dfmlr['significant']).astype(int).round(2)

dfmlr = dfmlr[['feature1','feature2','rho', 'site','target','range','pvalue','95ci_low','95ci_high', 'n', 'n0']]

def pmean(df):
    return scipy.stats.ttest_1samp(df, 0)[1]
tmpsmeanstat = dfmlr[['feature1','feature2','rho','target','range']].groupby(['feature1', 'feature2','target','range']).agg([np.nanmean, np.nanstd, pmean]).reset_index()
tmpsmeanstat.columns = ['feature1','feature2','target','range','rho','std', 'pvalue']
tmpsmeanstat['95ci_low'] = tmpsmeanstat['rho']-2*tmpsmeanstat['std']
tmpsmeanstat['95ci_high'] = tmpsmeanstat['rho']+2*tmpsmeanstat['std']
tmpsmeanstat['site'] = 'MEAN'
tmpsmeanstat = tmpsmeanstat.drop('std',axis=1)
tmpsmeanstat['pvalue'] = tmpsmeanstat['pvalue'].round(2)

tmps2 = pd.concat([dfmlr, tmpsmeanstat])
tmps2['95ci_min'] = tmps2[['95ci_low','95ci_high']].min(axis=1)
tmps2['95ci_max'] = tmps2[['95ci_low','95ci_high']].max(axis=1)
tmps3  = tmps2[['feature1','feature2', 'target']].drop_duplicates()

dict_lab = {'2157-6':'creatineKinase(HK)',
          '1920-8':'AST(HK)',      
          '2532-0':'LDH(HK)',
          '4542-7':'Haptoglobin(HK)', 
          '3084-1':'uricAcid(HK)',
        '2823-3':'potassium',
        '17861-6':'calcium',
        '2951-2':'sodium', 
        '2160-0':'sCr',
        'FLAG':'FLAG'}

dict_med = {'A07DA': 'DiphenoxylateLoperamide(LK)',
              'C03CA': 'FurosemideBumetanideTorsemide(LKLNa)',
              'C03CB': 'FurosemideBumetanideTorsemide(LKLNa)',
              'C03EB': 'FurosemideBumetanideTorsemide(LKLNa)',
              'C03AA': 'hydrochlorothiazide(LNa)',
              'C03AB': 'hydrochlorothiazide(LNa)',
              'C03AH': 'chlorothiazide(LNa)',
              'C03BA': 'chlorthalidone(LNa)',
              'C03EA': 'hydrochlorothiazide(LNa)',
              'C09DX': 'SacubitrilValsartan(HK)',
              'C03DA': 'eplerenone(HK)',
              'C03XA': 'tolvaptan(LNa)',
              'C09XA': 'hydrochlorothiazide(LNa)',
              'C09DX': 'hydrochlorothiazide(LNa)',
              'C03AX': 'hydrochlorothiazide(LNa)',
              'C09BX': 'hydrochlorothiazide(LNa)',
              'M05BA': 'PamidronateZoledronate(HCa)',
              'M05BB': 'etidronate(HCa)'}

plotdict = dict()

tmps3 = tmps3.replace(dict_lab)
tmps2 = tmps2.replace(dict_lab)

tmps3 = tmps3.replace(dict_med)
tmps2 = tmps2.replace(dict_med)      

tmps2

loinc2 = '2823-3'
loinc3 = 'FLAG'
plot_prefix = "plot3_"
for row in tmps3.iterrows():
    f1 = row[1][0]
    f2 = row[1][1]
    f3 = row[1][2]
    tmps2X = tmps2[(tmps2['feature1']==f1) & (tmps2['feature2']==f2) & (tmps2['target']==f3)]
    tmps2X  = tmps2X.reset_index(drop=True)
    fp.forestplot(tmps2X, 
              estimate="rho",  # col containing estimated effect size 
              ll="95ci_min", hl="95ci_max",  # columns containing conf. int. lower and higher limits
              varlabel="site",  # column containing variable label
              ylabel="Confidence interval",  # y-label title
              xlabel=f1+' vs '+f2+ '('+f3+')',
              groupvar="range",
              rightannote=["pvalue", "n", "n0"],
              right_annoteheaders=["pvalue", "n(cofounder)", "n(base)"])
    plt.savefig(plot_prefix+f1+'_'+f2+'_'+f3+"_.png", bbox_inches="tight")
    plotdict[(f1,f2,loinc2,loinc3)] = plt.gca()

tmps2X = tmps2[(tmps2['feature1']==f1) & (tmps2['feature2']==f2) & (tmps2['target']==f3)]

tmps2X  = tmps2X.reset_index(drop=True)

fp.forestplot(tmps2X, 
          estimate="rho",  # col containing estimated effect size 
          ll="95ci_min", hl="95ci_max",  # columns containing conf. int. lower and higher limits
          varlabel="site",  # column containing variable label
          ylabel="Confidence interval",  # y-label title
          xlabel=f1+' vs '+f2+ '('+f3+')',
          groupvar="range",
          rightannote=["pvalue", "n", "n0"],
          right_annoteheaders=["pvalue", "n(cofounder)", "n(base)"])
plt.savefig("plot3_"+f1+'_'+f2+'_'+dict_lab[loinc2]+'_'+dict_lab[loinc3]+".png", bbox_inches="tight")